## Enabling ML processes using Mosaics

With mosaic-json it's pretty easy to scale your (tile based) Machine learning processes

In [ ]:
import requests
import datetime
import urllib.parse
from concurrent import futures

from supermercado.burntiles import tile_extrema

endpoint = "https://landsatlive.live"

### Create the mosaic

In [ ]:
# Define AOI
bounds = [-6.17431640625, 42.79540065303723, 8.28369140625, 51.0275763378024]

# Date Filters
start = datetime.datetime.strptime("2013-01-01", "%Y-%m-%d").strftime("%Y-%m-%dT00:00:00Z")
end = datetime.datetime.strptime("2019-12-01", "%Y-%m-%d").strftime("%Y-%m-%dT23:59:59Z")

# STAC QUERY (sat-api)
query = {
    "bbox": bounds,
    "time": f"{start}/{end}",
    "query": {
        "eo:sun_elevation": {"gt": 0},
        "landsat:tier": {"eq": "T1"},
        "collection": {"eq": "landsat-8-l1"},
        "eo:cloud_cover": {"gte": 0, "lt": 5}  # Cloud Filters
    },
    "sort": [{
      "field": "eo:cloud_cover",
      "direction": "asc"
    }],
}

# We post the query to the mosaic endpoint
# `optimized_selection: True` is an option to force only one PATH-ROW scene per tile
# Because we will use `pixel_selection:"first"` this will minimized
# the number of image fetched for each tile request
results = requests.post(
    f"{endpoint}/mosaic/create", 
    json=query,
    params={
        "optimized_selection": True,
        "seasons": "spring"  # this should be a "," separated list of seasons
    },
).json()

### Edit the tile URL

In [ ]:
# See https://landsatlive.live/tiles/docs for the list of options you can pass to the tile url
query_params = dict(
    bands="4,3,2", # True Color RGB
    color_ops="gamma RGB 3.5, saturation 1.7, sigmoidal RGB 15 0.35",
    pixel_selection="first",
)
tiles_url = results["tiles"][0] + urllib.parse.urlencode(query_params)

### Get the list of mercator tiles

In [ ]:
# Get Mercator tiles covering the AOI
zoom = results["maxzoom"]
extrema = tile_extrema(bounds, zoom)
tiles = []
for x in range(extrema["x"]["min"], extrema["x"]["max"]):
    for y in range(extrema["y"]["min"], extrema["y"]["max"]):
        tiles.append(f"{zoom}-{x}-{y}")

print(len(tiles))

### Do something with the tiles

In [ ]:
def worker(tile):
    # Do Whatever you want here
    # e.g send tile to an inference pipeline
    pass


with futures.ThreadPoolExecutor() as executor:
    executor.map(worker, tiles)